# LangChain 实战：房产销售聊天机器人

## 使用 GPT-4 构造销售话术数据

ChatGPT 分享链接：https://chat.openai.com/share/f3e4b9b0-95fb-4c6a-a3c7-f901dd194c91


使用 ChatGPT 构造销售数据的 Prompt 示例：

```
你是中国顶级的房地产销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]

```

GPT-4 回复结果：

```
在房地产销售行业中，精心准备的销售话术可以极大地提高成交率。以下是一些适用于中国房地产销售的实用话术：

1.  
[客户问题] 这个小区交通便利吗？
[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。

2.  
[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

3.  
[客户问题] 我看房价还在涨，这个投资回报怎么样？
[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。

4.  
[客户问题] 有没有学校？
[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。

5.  
[客户问题] 物业管理怎么样？
[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。

6.  
[客户问题] 我想要南向的房子。
[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。

7.  
[客户问题] 这个小区安全吗？
[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。

8.  
[客户问题] 预计什么时候交房？
[销售回答] 根据目前的进度，我们预计将在明年底交房。

9.  
[客户问题] 我不想要一楼的房子。
[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。

10.  
[客户问题] 有优惠吗？
[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [real_estate_sales_data.txt](real_estate_sales_data.txt) 文件中

In [15]:
with open("real_estate_sales_data.txt", encoding='utf-8') as f:
    real_estate_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [16]:
from langchain.text_splitter import CharacterTextSplitter

In [17]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [18]:
docs = text_splitter.create_documents([real_estate_sales])

In [19]:
docs[0]

Document(page_content='[客户问题] 这个价格似乎比市场上的其他供应商要高。\n[销售回答] 我们的集装箱虽然价格稍高，但质量和持久性都经过严格测试，保证了长期使用的经济效益。')

In [20]:
len(docs)

100

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [26]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

In [27]:
query = "为什么要选择你们"

In [28]:
answer_list = db.similarity_search(query)

In [29]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 为什么我们要选择你们而不是其他竞争对手？
[销售回答] 我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。

[客户问题] 我们只是小规模业务，你们是否愿意与我们合作？
[销售回答] 我们提供灵活的定制服务，无论您的订单大小，我们都致力于提供最适合您需求的解决方案。

[客户问题] 我们只是小规模业务，你们是否愿意与我们合作？
[销售回答] 我们致力于支持各种规模的业务，无论大小规模，都可以进行沟通合作

[客户问题] 我们需要这么先进的集装箱吗？
[销售回答] 考虑到未来的市场竞争和效率提升，先进的设备可以帮助您保持领先地位。



In [30]:
db.save_local("real_estates_sale")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [31]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [32]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000022F7603ABF0>, search_kwargs={'k': 3})

In [33]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


[客户问题] 为什么我们要选择你们而不是其他竞争对手？
[销售回答] 我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。

[客户问题] 我们只是小规模业务，你们是否愿意与我们合作？
[销售回答] 我们提供灵活的定制服务，无论您的订单大小，我们都致力于提供最适合您需求的解决方案。

[客户问题] 我们只是小规模业务，你们是否愿意与我们合作？
[销售回答] 我们致力于支持各种规模的业务，无论大小规模，都可以进行沟通合作



In [34]:
docs = topK_retriever.get_relevant_documents("你们有没有售后赔付服务？")

In [35]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 贵公司提供售后服务吗？
[销售回答] 我们提供全面的售后服务，包括保修、维修和定期检查等，确保您的集装箱始终处于最佳状态。

[客户问题] 如果我们在海外，你们的售后服务怎么保障？
[销售回答] 我们有全球化的服务网络，无论您在哪里，我们都能提供迅速的售后支持。此外，我们也提供在线技术支持和备件快速配送。

[客户问题] 如果出现质量问题，你们如何处理？
[销售回答] 我们非常重视产品质量，并提供保修服务。如果在保修期内发现任何质量问题，我们将免费为您进行维修或更换。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [36]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [37]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 为什么我们要选择你们而不是其他竞争对手？
[销售回答] 我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。



### 提取向量数据库中的`销售回答`

In [38]:
docs = retriever.get_relevant_documents(query)

In [39]:
docs[0].page_content

'[客户问题] 为什么我们要选择你们而不是其他竞争对手？\n[销售回答] 我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。'

In [40]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 为什么我们要选择你们而不是其他竞争对手？\n', '我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。']

In [41]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [42]:
ans

'我们公司不仅提供高质量的产品，我们还强调客户服务和满意度。'

#### 尝试各种问题

In [43]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [44]:
query = "有便宜点的箱子吗"

print(sales(query))

[]


D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [45]:
print(sales(query, 0.75))

['是的，我们了解有些客户可能需要短期使用集装箱。', '我们的集装箱虽然价格稍高，但质量和持久性都经过严格测试，保证了长期使用的经济效益。', '我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。', '是的，我们为客户提供便利的集装箱存储服务，您可以选择在我们的设施中暂时存储集装箱。']


In [46]:
query = "有便宜点的箱子吗"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\vectorstores.py:342: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: ['是的，我们了解有些客户可能需要短期使用集装箱。', '我们的集装箱虽然价格稍高，但质量和持久性都经过严格测试，保证了长期使用的经济效益。', '我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。', '是的，我们为客户提供便利的集装箱存储服务，您可以选择在我们的设施中暂时存储集装箱。']

score:0.5 ans: ['是的，我们了解有些客户可能需要短期使用集装箱。', '我们的集装箱虽然价格稍高，但质量和持久性都经过严格测试，保证了长期使用的经济效益。', '我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。', '是的，我们为客户提供便利的集装箱存储服务，您可以选择在我们的设施中暂时存储集装箱。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [47]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4-1106-preview", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [48]:
qa_chain({"query": "你们有哪些类型的箱子？"})

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


{'query': '你们有哪些类型的箱子？',
 'result': '我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。'}

In [49]:
qa_chain({"query": "你们有哪些类型的箱子？"})

{'query': '你们有哪些类型的箱子？',
 'result': '我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。'}

In [51]:
print(sales("你们有哪些类型的箱子？"))

['我们提供多种类型的集装箱，包括干货箱、冷藏箱、开顶箱、平板箱等，以适应不同客户的多样化需求。', '我们有多种类型的集装箱，可以满足不同重量级货物的运输。']


## 加载 FAISS 向量数据库已有结果

In [53]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings(), allow_dangerous_deserialization=True)
 

In [54]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

D:\software\Miniconda\envs\langchain\lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [55]:
qa_chain({"query": "有哪些支付方式"})

{'query': '有哪些支付方式', 'result': '我们提供的付款方式有电汇、信用卡、支票以及通过支付宝或微信支付等。'}

In [37]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [57]:
qa_chain({"query": "有哪些支付方式"})

{'query': '有哪些支付方式', 'result': '我们提供的支付方式包括电汇、信用卡、支票以及通过支付宝或微信支付等。'}

In [58]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [59]:
result = qa_chain({"query": "有哪些支付方式"})

In [60]:
result

{'query': '有哪些支付方式',
 'result': '我们提供的支付方式有电汇、信用卡、支票以及通过支付宝或微信支付等。',
 'source_documents': [Document(page_content='[客户问题] 你们提供哪些付款方式？\n[销售回答] 为了方便不同客户的需要，我们提供多种付款方式，包括电汇、信用卡、支票以及通过支付宝或微信支付等。')]}